In [1]:
"""
Unfold velocity and output file as cfradial
"""
import os
from glob import glob

import pyart
import openradartools as ort
from unravel import dealias



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data/en0/jss548/miniconda3/envs/openradar/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
source_folder = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset'
dest_folder = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset_level1b'

In [5]:
odim_ffn_list = sorted(glob(source_folder + '/*.h5'))
for odim_ffn in odim_ffn_list:
    print(odim_ffn)
    radar = pyart.aux_io.read_odim_h5(odim_ffn)
    nyquist_list = ort.vel.extract_nyquist(radar, odim_ffn)  # returns empty list if something fails
    print(list(zip(radar.fixed_angle['data'],nyquist_list)))

    gatefilter = pyart.correct.GateFilter(radar)
    vdop_unfold = dealias.unravel_3D_pyart(
        radar, 'velocity_horizontal', 'reflectivity_horizontal',
        gatefilter=gatefilter,
        nyquist_velocity=nyquist_list,
        
    )
    radar.add_field_like('velocity_horizontal','corrected_velocity', vdop_unfold, replace_existing=True)
    radar.fields['corrected_velocity']['dealiasing_correction'] = 'UNRAVEL dealiasing algorithm developed by ' \
                                                                    'Louf et al. (2020) ' \
                                                                    'doi:10.1175/JTECH-D-19-0020.1'
    out_ffn = f'{dest_folder}/{os.path.basename(odim_ffn)[:-3]}_unravel.nc'
    pyart.io.write_cfradial(out_ffn, radar, format='NETCDF4')


/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_24_ODIMH5_PVOL6S_VOL_CASCV.h5
[(24.4, 31.746000289916992), (20.2, 31.746000289916992), (16.6, 31.746000289916992), (13.7, 31.746000289916992), (11.3, 31.746000289916992), (9.4, 31.746000289916992), (7.7, 23.809499740600586), (6.4, 23.809499740600586), (5.3, 16.402099609375), (4.4, 16.402099609375), (3.5, 16.402099609375), (2.7, 47.61899948120117), (2.1, 47.61899948120117), (1.6, 47.61899948120117), (1.2, 47.61899948120117), (0.8, 47.61899948120117), (0.4, 47.61899948120117)]
